In [10]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection


# Lựa chọn đặc trưng

### (1) Tạo thư mục lưu trữ (nếu chưa có)

In [11]:
save_dir=f"{exps_dir}/exp1/exp_gan"

os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [12]:
x_train=pd.read_excel(f'{save_dir}/x_train.xlsx')
y_train=pd.read_excel(f'{save_dir}/y_train.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(1020, 39) (1020, 1) (330, 39) (330, 1)


### (2) Lựa chọn các đặc trưng

In [13]:
x_train

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
0,0,2,11,1,2,-0.135115,0,1,6,2,9,1,-0.940368,0.809323,3,0,3,3,27,52,1,0,1,2,1,2,-1.676803,-1.182425,-1.288254,-1.729038,11,19,2,-1.423969,-0.446155,0.986569,-1.174354,0.998447,0
1,1,1,12,0,1,0.495180,0,0,2,10,3,2,0.655758,0.809323,2,1,1,1,52,19,2,0,1,2,1,1,0.337853,-0.064942,-0.211904,0.310913,13,28,1,-0.862658,1.249789,-1.531699,1.432852,0.998447,0
2,1,2,12,2,0,0.341022,0,1,4,8,18,5,-0.940368,0.809323,2,1,0,1,49,106,0,0,1,1,1,1,0.774402,0.034155,-0.116454,0.822176,4,30,0,-0.413609,-1.576785,-1.657613,0.979425,-0.908699,0
3,1,0,11,2,0,0.005199,0,1,2,11,0,3,-0.940368,-0.503457,2,1,2,2,26,126,0,0,1,2,0,2,0.775198,1.195916,1.002544,0.260909,5,16,1,-0.189085,0.684474,-1.783526,-0.947640,-0.908699,0
4,0,2,36,0,2,-0.009186,5,0,3,4,16,5,-0.940368,-0.947582,0,2,0,1,49,52,0,1,2,1,0,1,0.237479,1.136879,-0.268768,-0.094220,5,14,2,1.831635,-0.163498,0.860655,0.185927,0.998447,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,0,2,12,1,0,0.623489,3,0,5,12,8,0,0.598539,0.809323,2,3,2,3,49,147,0,0,2,2,3,1,1.717204,2.102553,1.875808,1.028824,0,31,2,-1.199445,-1.859442,0.608828,0.525998,-0.908699,0
1016,1,2,12,2,0,-1.738381,5,1,2,6,15,4,0.670816,0.809323,2,3,2,3,52,105,1,0,1,0,2,2,1.195815,1.600740,-0.045374,0.965044,11,21,2,0.035440,0.967132,-0.020739,-0.720927,-0.908699,0
1017,1,0,4,2,0,1.655908,0,1,2,2,16,2,-0.940368,0.809323,0,3,0,0,8,36,0,2,1,2,1,2,-0.262800,-0.386482,1.519598,-1.525451,9,29,2,0.484489,1.249789,-0.020739,1.432852,0.998447,0
1018,1,0,11,2,2,1.255443,0,1,5,11,8,2,0.198001,0.809323,2,1,1,1,43,52,1,0,1,0,3,0,0.650925,0.871213,0.689793,0.258868,5,14,1,1.494849,-1.294127,-1.783526,0.866068,-0.908699,0


In [14]:
def modelcheck_info(model,name):
    print(name)
    try:
        coefficients = model.coef_.flatten()
    except:
        coefficients = model.feature_importances_.flatten()
    feature_importance = pd.DataFrame({
        'Feature': x_train.columns,
        'Coefficient': coefficients
    })

    print('bi loai bo',feature_importance[feature_importance['Coefficient'] == 0])
    feature_importance = feature_importance[feature_importance['Coefficient'] != 0]
    feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
    feature_importance.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)

    print("Xếp hạng các đặc trưng theo mức độ quan trọng:")
    print(feature_importance[['Feature', 'Coefficient']])

In [15]:
models=[]
models.append(LogisticRegression())
models.append(LinearDiscriminantAnalysis())
models.append(DecisionTreeClassifier())
names=['LR','LDA','DTC']
feature_SelectFromModel={}
for model,name in zip(models,names):
    check = model.fit(x_train, y_train)
    selector = SelectFromModel(check, prefit=True)

    selected_features = x_train.columns[(selector.get_support())]
    modelcheck_info(model,name)
    print("Các đặc trưng được chọn:")
    feature_SelectFromModel[name]=list(selected_features)
    print(selected_features)

LR
bi loai bo                Feature  Coefficient
38  year_incident_date          0.0
Xếp hạng các đặc trưng theo mức độ quan trọng:
                        Feature  Coefficient
16            incident_severity    -0.904988
36            day_incident_date    -0.606213
26           total_claim_amount    -0.559480
35        year_policy_bind_date     0.543563
29                vehicle_claim     0.413736
27                 injury_claim    -0.413618
34       month_policy_bind_date     0.316815
5         policy_annual_premium    -0.273034
1                           age     0.258234
28               property_claim     0.236686
7                   insured_sex     0.214062
6                umbrella_limit     0.173410
20     incident_hour_of_the_day     0.163457
13                 capital-loss     0.162330
33         day_policy_bind_date    -0.136838
22              property_damage     0.121103
23              bodily_injuries     0.109337
15               collision_type     0.100905
37          

In [18]:
label_encoder = LabelEncoder()
y_train['fraud_reported'] = label_encoder.fit_transform(y_train['fraud_reported'])

### Giải thích:

Phương pháp **SelectFromModel** từ thư viện sklearn.feature_selection. Phương pháp này hoạt động bằng cách sử dụng một mô hình học máy để đánh giá độ quan trọng của từng đặc trưng và sau đó chọn những đặc trưng quan trọng nhất.

   Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
  
       #### 1. LogisticRegression:
       Các đặc trưng được chọn:
* Index(['age', 'policy_csl', 'insured_education_level', 'insured_occupation',
       'incident_severity', 'incident_state', 'incident_city',
       'incident_hour_of_the_day', 'property_damage', 'witnesses',
       'property_claim', 'vehicle_claim', 'auto_make'],
      dtype='object')
      
   #### 2. LinearDiscriminantAnalysis:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip', 'insured_sex',
       'incident_severity', 'incident_hour_of_the_day', 'property_damage',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')
      
   #### 3. DecisionTreeClassifier:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip',
       'insured_occupation', 'insured_hobbies', 'incident_severity',
       'incident_location', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'month_policy_bind_date', 'day_incident_date'],
      dtype='object')

In [16]:
def feature_selection_inRFE(model,name):
    rfe=RFE(estimator=model, n_features_to_select=10)
    fit=rfe.fit(x_train, y_train)
#     print("Num Features: %d"% fit.n_features_)
#     print("Selected Features: %s" % fit.support_)
#     print("Features Ranking: %s" %fit.ranking_)
#     for index, value in enumerate(fit.ranking_):
#         print(f"{value}: {x_train.columns[index]}")
    sorted_columns = [x for _, x in sorted(zip(fit.ranking_, x_train.columns))]
    sort=sorted(fit.ranking_)

#     for i,j in zip(sort,sorted_columns):
#         print("Columns sorted by RFE ranking",i,":",j)

    return [j for i,j in zip(sort,sorted_columns) if i==1]

In [19]:
models = [
    LogisticRegression(),
    LinearDiscriminantAnalysis(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LGBMClassifier(),  
    XGBClassifier(use_label_encoder=False)  
]
feature_RFE={}

names = ['LR', 'LDA', 'DTC', 'RD', 'LGBM', 'XGB']
for model,name in zip(models,names):
    print("RFE")
    print(f"Kiểm tra mô hình {name}")
    model.fit(x_train,y_train)
    print(feature_selection_inRFE(model,name))
    feature_RFE[name]=feature_selection_inRFE(model,name)

RFE
Kiểm tra mô hình LR
['day_incident_date', 'day_policy_bind_date', 'incident_severity', 'injury_claim', 'month_policy_bind_date', 'policy_annual_premium', 'property_claim', 'total_claim_amount', 'vehicle_claim', 'year_policy_bind_date']
RFE
Kiểm tra mô hình LDA
['day_incident_date', 'incident_severity', 'injury_claim', 'insured_sex', 'month_policy_bind_date', 'policy_annual_premium', 'property_claim', 'total_claim_amount', 'vehicle_claim', 'year_policy_bind_date']
RFE
Kiểm tra mô hình DTC
['capital-loss', 'day_incident_date', 'incident_severity', 'incident_state', 'injury_claim', 'insured_hobbies', 'policy_state', 'property_claim', 'total_claim_amount', 'year_policy_bind_date']
RFE
Kiểm tra mô hình RD
['capital-gains', 'day_incident_date', 'incident_severity', 'incident_state', 'injury_claim', 'insured_hobbies', 'policy_annual_premium', 'policy_state', 'total_claim_amount', 'year_policy_bind_date']
RFE
Kiểm tra mô hình LGBM
[LightGBM] [Info] Number of positive: 509, number of negati

### Giải thích:
**Recursive Feature Elimination (RFE)** là một phương pháp trong lựa chọn đặc trưng (feature selection) được sử dụng trong học máy để giảm số lượng đặc trưng của mô hình mà vẫn giữ được hiệu suất tốt.

Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
   * **'RD'**: RandomForestClassifier

In [20]:
special_models=[KNeighborsClassifier(),GaussianNB(),MLPClassifier(),GradientBoostingClassifier(),MultinomialNB(),SVC()]
names=['KNC','GNB','NN','BG','NB','SVM']
def feature_selection_inKBest(model, name):
    k_best = SelectKBest(score_func=f_classif, k=11)
    fit = k_best.fit(x_train, y_train)
    selected_features = fit.get_support()
    selected_column_names = x_train.columns[selected_features]
    return selected_column_names
feature_Kbest={}
for model,name in zip(special_models,names):
    print(f'Lua chon dac trung {name}')
    print(feature_selection_inKBest(model, name))
    feature_Kbest[name]=list(feature_selection_inKBest(model, name))

Lua chon dac trung KNC
Index(['policy_state', 'policy_annual_premium', 'capital-gains', 'incident_severity', 'incident_state', 'total_claim_amount', 'injury_claim', 'day_policy_bind_date', 'month_policy_bind_date', 'year_policy_bind_date', 'day_incident_date'], dtype='object')
Lua chon dac trung GNB
Index(['policy_state', 'policy_annual_premium', 'capital-gains', 'incident_severity', 'incident_state', 'total_claim_amount', 'injury_claim', 'day_policy_bind_date', 'month_policy_bind_date', 'year_policy_bind_date', 'day_incident_date'], dtype='object')
Lua chon dac trung NN
Index(['policy_state', 'policy_annual_premium', 'capital-gains', 'incident_severity', 'incident_state', 'total_claim_amount', 'injury_claim', 'day_policy_bind_date', 'month_policy_bind_date', 'year_policy_bind_date', 'day_incident_date'], dtype='object')
Lua chon dac trung BG
Index(['policy_state', 'policy_annual_premium', 'capital-gains', 'incident_severity', 'incident_state', 'total_claim_amount', 'injury_claim', 'da

### Giải thích:
Phương pháp **SelectKBest** từ thư viện sklearn.feature_selection để lựa chọn đặc trưng dựa trên kỹ thuật kiểm định thống kê.
 
Các model được dùng trong phương pháp này:
   * **'KNC'**: KNeighborsClassifier
   * **'GNB'**: GaussianNB
   * **'NN'**: MLPClassifier
   * **'BG'**: GradientBoostingClassifier
   * **'NB'**: MultinomialNB
   * **'SVM'**: SVC

In [21]:
print(feature_Kbest)
print(feature_SelectFromModel)
print(feature_RFE)

{'KNC': ['policy_state', 'policy_annual_premium', 'capital-gains', 'incident_severity', 'incident_state', 'total_claim_amount', 'injury_claim', 'day_policy_bind_date', 'month_policy_bind_date', 'year_policy_bind_date', 'day_incident_date'], 'GNB': ['policy_state', 'policy_annual_premium', 'capital-gains', 'incident_severity', 'incident_state', 'total_claim_amount', 'injury_claim', 'day_policy_bind_date', 'month_policy_bind_date', 'year_policy_bind_date', 'day_incident_date'], 'NN': ['policy_state', 'policy_annual_premium', 'capital-gains', 'incident_severity', 'incident_state', 'total_claim_amount', 'injury_claim', 'day_policy_bind_date', 'month_policy_bind_date', 'year_policy_bind_date', 'day_incident_date'], 'BG': ['policy_state', 'policy_annual_premium', 'capital-gains', 'incident_severity', 'incident_state', 'total_claim_amount', 'injury_claim', 'day_policy_bind_date', 'month_policy_bind_date', 'year_policy_bind_date', 'day_incident_date'], 'NB': ['policy_state', 'policy_annual_pre

In [22]:
feature_model_selection={**feature_RFE,**feature_Kbest}
np.savez(f'{save_dir}/feature_model_selection.npz', feature_model_selection = feature_model_selection)
feature_model_selection

{'LR': ['day_incident_date',
  'day_policy_bind_date',
  'incident_severity',
  'injury_claim',
  'month_policy_bind_date',
  'policy_annual_premium',
  'property_claim',
  'total_claim_amount',
  'vehicle_claim',
  'year_policy_bind_date'],
 'LDA': ['day_incident_date',
  'incident_severity',
  'injury_claim',
  'insured_sex',
  'month_policy_bind_date',
  'policy_annual_premium',
  'property_claim',
  'total_claim_amount',
  'vehicle_claim',
  'year_policy_bind_date'],
 'DTC': ['day_incident_date',
  'incident_severity',
  'incident_state',
  'injury_claim',
  'insured_hobbies',
  'insured_occupation',
  'month_policy_bind_date',
  'policy_annual_premium',
  'policy_state',
  'year_policy_bind_date'],
 'RD': ['day_incident_date',
  'incident_severity',
  'incident_state',
  'injury_claim',
  'insured_hobbies',
  'policy_annual_premium',
  'policy_state',
  'property_claim',
  'total_claim_amount',
  'year_policy_bind_date'],
 'LGBM': ['day_incident_date',
  'day_policy_bind_date',
  

* gom nhóm thông tin về các đặc trưng được chọn từ hai phương pháp lựa chọn đặc trưng khác nhau (RFE và SelectKBest) vào từ điển **feature_model_selection**.

In [23]:
dict(np.load(f'{save_dir}/feature_model_selection.npz',allow_pickle=True))['feature_model_selection']

array({'LR': ['day_incident_date', 'day_policy_bind_date', 'incident_severity', 'injury_claim', 'month_policy_bind_date', 'policy_annual_premium', 'property_claim', 'total_claim_amount', 'vehicle_claim', 'year_policy_bind_date'], 'LDA': ['day_incident_date', 'incident_severity', 'injury_claim', 'insured_sex', 'month_policy_bind_date', 'policy_annual_premium', 'property_claim', 'total_claim_amount', 'vehicle_claim', 'year_policy_bind_date'], 'DTC': ['day_incident_date', 'incident_severity', 'incident_state', 'injury_claim', 'insured_hobbies', 'insured_occupation', 'month_policy_bind_date', 'policy_annual_premium', 'policy_state', 'year_policy_bind_date'], 'RD': ['day_incident_date', 'incident_severity', 'incident_state', 'injury_claim', 'insured_hobbies', 'policy_annual_premium', 'policy_state', 'property_claim', 'total_claim_amount', 'year_policy_bind_date'], 'LGBM': ['day_incident_date', 'day_policy_bind_date', 'incident_state', 'injury_claim', 'insured_hobbies', 'month_policy_bind_da